In [1]:
from matplotlib import pyplot as plt
import errno

plt.style.use('seaborn-whitegrid')
plt.rcParams.update({'figure.max_open_warning': 0})

import numpy as np
import os

import threading

In [2]:
def plot_cwnd_vs_time_individual(noOfSources = 2, rtt = "10ms" , flavour= "TcpBbr"):
    
    f = plt.figure()
    f.set_figwidth(25)
    f.set_figheight(6)
    
    for i in range(noOfSources):
        cwndvstime = {}

        with open(f"/mnt/Store/Project-summer/runtime/{flavour}/{rtt}/{noOfSources}/cwndDropTail_{i}.txt", "r") as f:
            lines = f.readlines()
        
        for line in lines:
            split_line = line.split()
            cwndvstime[float(split_line[0])] = float(split_line[1]);

        plt.plot(cwndvstime.keys(), cwndvstime.values())
            
    plt.xlabel("time")
    plt.ylabel("cwnd")
    plt.title(f"Congestion windows size vs time     Sources:{noOfSources}    RTT:{rtt}    Flavour:{flavour}")
    f.close()
     
def plot_cwnd_vs_time(noOfSources = 2, rtt = "10ms" , flavour= "TcpBbr"):
    
    f = plt.figure()
    f.set_figwidth(25)
    f.set_figheight(6)
    source_cwnd = {}
    
    for i in range(noOfSources):
        cwndvstime = {}

        with open(f"/mnt/Store/Project-summer/runtime/{flavour}/{rtt}/{noOfSources}/cwndDropTail_{i}.txt", "r") as f:
            lines = f.readlines()
        
        for line in lines:
            split_line = line.split()
            cwndvstime[float(split_line[0])] = float(split_line[1]);

        source_cwnd[i] = cwndvstime; 
    
    start_time = int(list(source_cwnd[0].keys())[0])
    
    cwnd_vs_time_avg = {}
    
    for t in range(start_time, start_time+len(source_cwnd[0])):
        cwnd_vs_time_avg[t] = 0;
        for i in range(noOfSources):
            cwnd_vs_time_avg[t] += source_cwnd[i][t];
        cwnd_vs_time_avg[t] = cwnd_vs_time_avg[t] / noOfSources;
    
    plt.plot(cwnd_vs_time_avg.keys(), cwnd_vs_time_avg.values() );
    plt.xlabel("time")
    plt.ylabel("avg cwnd")
    plt.title(f"Average Congestion windows size vs time     Sources:{noOfSources}    RTT:{rtt}    Flavour:{flavour}")
    f.close()
    
def plot_dropped_vs_time( noOfSources = 2, rtt = "10ms" , flavour= "TcpBbr"):
    f = plt.figure()
    f.set_figwidth(25)
    f.set_figheight(6)
    
    
    droppedpvstime = {}
    
    with open(f"/mnt/Store/Project-summer/runtime/{flavour}/{rtt}/{noOfSources}/droppedPacketTrace.txt", "r") as f:
        lines = f.readlines()
        prev = 0;
        total_dropped = 0;
        bin = 0;
        for line in lines:
            split_line = line.split()
            
            if ( float(split_line[0]) - prev < 0.1) :
                bin = bin+1;
            else:
                droppedpvstime[prev] = bin;
                prev = float(split_line[0]);
                bin = 0;

    
    plt.plot(droppedpvstime.keys(), droppedpvstime.values())
    
    plt.xlabel("time")
    plt.ylabel("Packet Dropped")
    plt.title(f"Packet Dropped vs time     Sources:{noOfSources}    RTT:{rtt}    Flavour:{flavour}")
    f.close()

def plot_link_utilization_vs_time( noOfSources = 2, rtt = "10ms" , flavour= "TcpBbr", maxQSize = "2084p", bottleneck_bandwidth = 100 , packetSize = 1458):
    f = plt.figure()
    f.set_figwidth(25)
    f.set_figheight(6)
    
    
    linkutilvstime = {}

    with open(f"/mnt/Store/Project-summer/runtime/{flavour}/{rtt}/{noOfSources}/bottleneckTx.txt", "r") as f:
        lines = f.readlines()
        
    prev_packetsTx = 0;
    
    for line in lines:
        split_line = line.split()
        linkutilvstime[float(split_line[0])] = 100 * (8*(float(split_line[1])-float(prev_packetsTx))*packetSize)/(bottleneck_bandwidth*1024*1024);
        prev_packetsTx = split_line[1];

    
    
    plt.plot(linkutilvstime.keys(), linkutilvstime.values())
            
    plt.xlabel("time")
    plt.ylabel("Link utilization %")
    plt.title(f"Link utilization vs time     Sources:{noOfSources}    RTT:{rtt}    Flavour:{flavour}")
    f.close()

def plot_qsize_vs_time( noOfSources = 2, rtt = "10ms" , flavour= "TcpBbr", maxQSize = "2084p"):
    f = plt.figure()
    f.set_figwidth(25)
    f.set_figheight(6)
    
    
    qutilvstime = {}

    with open(f"/mnt/Store/Project-summer/runtime/{flavour}/{rtt}/{noOfSources}/qsizeTrace.txt", "r") as f:
        lines = f.readlines()
    
    for line in lines:
        split_line = line.split()
        
        qutilvstime[float(split_line[0])] = float(split_line[1]);

    
    plt.plot(qutilvstime.keys(), qutilvstime.values())
            
    plt.xlabel("time")
    plt.ylabel("Q utilization (abs)")
    plt.title(f"Q utilization vs time     Sources:{noOfSources}    RTT:{rtt}    Flavour:{flavour}")
    f.close()

def clean(flavor = "default" , 
          time = 200 , 
          queueL = 5, 
          noOfSource = 4, 
          total_input_bandwidth = 2, bottleneck_bandwidth = 1, RTT = 10, recvWindow = 50):
    os.system("rm -rf {}_{}_{}".format(flavor, noOfSource, RTT));
    

In [3]:
def main(flavour = "TcpBbr" , time = 200 ,total_access_bandwidth = 120, bottleneck_bandwidth = 100, 
         bottleneck_delay = 1 , RTT = "10ms", packetSize = 1458, queueL = "5p", noOfSource = 2, 
         rcvBuff = 1458*10000, sndBuff = 1458*10000, enableBottleneckTrace = 0):
    
     # access_link_delay_val = RTT[0:-2]
     access_link_bandwidth_val = total_access_bandwidth[0:-4]
     
     #access_link_delay = str(float(access_link_delay_val)/4)+"ms"
     access_link_bandwidth = str(float(access_link_bandwidth_val)/noOfSource)+"Mbps"
     
     lock = threading.Lock();
     
     
     lock.acquire();   
     # Creating new directory structure to isolate 2 different processes and their output
     try: 
          if os.path.exists(f"/mnt/Store/Project-summer/runtime/{flavour}") == False:
               os.mkdir( f"/mnt/Store/Project-summer/runtime/{flavour}")
               
          if os.path.exists(f"/mnt/Store/Project-summer/runtime/{flavour}/{RTT}") == False:
               os.mkdir( f"/mnt/Store/Project-summer/runtime/{flavour}/{RTT}")
               
          if os.path.exists(f"/mnt/Store/Project-summer/runtime/{flavour}/{RTT}/{noOfSource}") == False:
               os.mkdir( f"/mnt/Store/Project-summer/runtime/{flavour}/{RTT}/{noOfSource}")
     except OSError as exc:
          if exc.errno != errno.EEXIST:
               raise
          pass
     
     lock.release();
    
     # if os.path.exists("/mnt/Store/Project-summer/runtime"):
     #     os.system("rm -rf /mnt/Store/Project-summer/runtime/*")
          
     cmd = f'''./ns3 run \"scratch/tcp_ftp_n --nSources={noOfSource} 
     --flavour={flavour} --bottleneckBandwidth={bottleneck_bandwidth} 
     --bottleneckDelay={bottleneck_delay} --accessBandwidth={access_link_bandwidth} 
     --queueSize={queueL} --pktSize={packetSize} 
     --simDuration={time} --rcvBuff={rcvBuff} --sndBuff={sndBuff} --enableBottleneckTrace={enableBottleneckTrace} 
     --RTT={RTT}\"'''
     
     #print(cmd)
     os.system(cmd)
     plot_cwnd_vs_time(noOfSource, RTT, flavour)
     plot_cwnd_vs_time_individual(noOfSource, RTT, flavour)
     plot_qsize_vs_time(noOfSource, RTT, flavour, queueL)
     plot_dropped_vs_time(noOfSource, RTT, flavour)
     plot_link_utilization_vs_time(noOfSource, RTT, flavour, bottleneck_bandwidth, packetSize)
     

In [4]:
# Running multiple simulations in-parallel using multi-threading
threads = []

pktSize = 1500
flavour = ["TcpBbr"]  
RTTs = ["10ms"]
sources = [10]


for rtt in RTTs:
    for s in sources:
        for flav in flavour:
            threads.append( threading.Thread ( target = main , args=(flav ,  200 , "120Mbps",  "100Mbps", "1ms" , rtt , pktSize,  "2084p", s, str(pktSize*10000), str(pktSize*10000), 0 )));
        
        
for thread in threads:
    thread.start()

for thread in threads:
    thread.join()

[  0%] Building CXX object CMakeFiles/stdlib_pch.dir/build-support/empty.cc.o


c++: internal compiler error: Segmentation fault signal terminated program cc1plus
Please submit a full bug report,
with preprocessed source if appropriate.
See <file:///usr/share/doc/gcc-11/README.Bugs> for instructions.
gmake[3]: *** [CMakeFiles/stdlib_pch.dir/build.make:93: CMakeFiles/stdlib_pch.dir/build-support/empty.cc.o] Error 4
gmake[2]: *** [CMakeFiles/Makefile2:3069: CMakeFiles/stdlib_pch.dir/all] Error 2
gmake[1]: *** [CMakeFiles/Makefile2:15667: scratch/CMakeFiles/scratch_tcp_ftp_n.dir/rule] Error 2
gmake: *** [Makefile:5250: scratch_tcp_ftp_n] Error 2
Exception in thread Thread-5 (main):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_6854/3961935257.py", line 44, in main
  File "/tmp/ipykernel_6854/3774131475.py", line 34, in plot_cwnd_vs_time
FileNotFoundError: [Errno 2

<Figure size 1800x432 with 0 Axes>

In [5]:
pktSize = 1500
main(flavour = "TcpNewReno", 
     time = 200, 
     noOfSource = 40, 
     total_access_bandwidth = "120Mbps", 
     RTT = "200ms",
     bottleneck_bandwidth = "100Mbps", 
     bottleneck_delay = "1ms", 
     packetSize = pktSize, 
     queueL = "2084p", 
     rcvBuff = str(pktSize*10000),
     sndBuff = str(pktSize*10000),
     enableBottleneckTrace = 0
     )

[  0%] Building CXX object CMakeFiles/stdlib_pch.dir/build-support/empty.cc.o


c++: internal compiler error: Segmentation fault signal terminated program cc1plus
Please submit a full bug report,
with preprocessed source if appropriate.
See <file:///usr/share/doc/gcc-11/README.Bugs> for instructions.
gmake[3]: *** [CMakeFiles/stdlib_pch.dir/build.make:93: CMakeFiles/stdlib_pch.dir/build-support/empty.cc.o] Error 4
gmake[2]: *** [CMakeFiles/Makefile2:3069: CMakeFiles/stdlib_pch.dir/all] Error 2
gmake[1]: *** [CMakeFiles/Makefile2:15667: scratch/CMakeFiles/scratch_tcp_ftp_n.dir/rule] Error 2
gmake: *** [Makefile:5250: scratch_tcp_ftp_n] Error 2


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/Store/Project-summer/runtime/TcpNewReno/200ms/40/cwndDropTail_0.txt'

<Figure size 1800x432 with 0 Axes>

the maths of internet cong control - R shrikanth

In [ ]:
pktSize = 1500
main(flavour = "TcpVegas", 
     time = 200, 
     noOfSource = 40, 
     total_access_bandwidth = "120Mbps", 
     RTT = "200ms",
     bottleneck_bandwidth = "100Mbps", 
     bottleneck_delay = "1ms", 
     packetSize = pktSize, 
     queueL = "2084p", 
     rcvBuff = str(pktSize*10000),
     sndBuff = str(pktSize*10000),
     enableBottleneckTrace = 0
     )